# Examples: `discrete` 

In [ ]:
import importlib
import sys
sys.path.append('..')

from ipywidgets import interact
import numpy as np
import matplotlib.pyplot as plt
import proplot as pplt
import psdist as ps
import psdist.visualization as psv

In [ ]:
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'
pplt.rc['figure.facecolor'] = 'white'
pplt.rc['grid'] = False

In [ ]:
n = 6
k = 10000
X = np.zeros((1, n))

state = np.random.RandomState(1950)
_Sigma = np.identity(n)
for i in range(n):
    for j in range(i):
        _Sigma[i, j] = _Sigma[j, i] = state.uniform(-0.2, 0.2)
X = np.vstack([X, state.multivariate_normal(np.zeros(n), _Sigma, size=k)])
for _ in range(4):
    scale = state.uniform(0.5, 1.5, size=n)
    loc = state.uniform(-3.0, 3.0, size=n)
    X = np.vstack([X, state.normal(loc=loc, scale=scale, size=(k, n))])
X = X - np.mean(X, axis=0)

In [ ]:
dims = ["x", "px", "y", "py", "z", "pz"]

## Corner plot 

In [ ]:
# alpha = 0.5
# cmaps = [pplt.Colormap(name, left=0.2) for name in ['blues', 'reds']]
# diag_colors = [cmap(0.75) for cmap in cmaps]
# diag_kind = 'step'

# axes = psp.corner(
#     data=X, 
#     bins='auto',
#     kind='hist',
#     diag_kind=diag_kind,
#     labels=dims,
#     diag_kws=dict(color=diag_colors[0], alpha=alpha),
#     cmap=cmaps[0],
#     alpha=alpha,
#     rms_ellipse=True,
#     rms_ellipse_kws=dict(level=2.0, color=diag_colors[0], alpha=0.5),
# )

# R = ap.phase_adv_mat(np.radians(45.0), np.radians(20.0), np.radians(10.0))
# axes = psp.corner(
#     data=(psb.apply(R, X) + state.uniform(-3.0, 3.0, size=(1, X.shape[1]))),
#     axes=axes,
#     diag_kind=diag_kind,
#     diag_kws=dict(color=diag_colors[1], alpha=alpha),
#     autolim_kws=dict(),
#     cmap=cmaps[1],
#     alpha=alpha,
#     rms_ellipse=True,
#     rms_ellipse_kws=dict(level=2.0, color=diag_colors[1], alpha=0.5),
# )

## Interactive slicing

In [ ]:
importlib.reload(psv.discrete)

In [ ]:
kws = dict(
    cmap=pplt.Colormap('blues', left=0.05),
    profx=True, profy=True,
    prof_kws=dict(color='black'),
    process_kws=dict(norm='max'),
)
psv.discrete.plot2d_interactive_slice(X, dims=dims, **kws)

## 2D plotting

In [ ]:
axis = (0, 1)
X = ps.discrete.downsample(X, 1000)

In [ ]:
cmap = pplt.Colormap('blues', left=0.05, right=1.0)

fig, axs = pplt.subplots(ncols=3, figwidth=6.0)
for ax, kind in zip(axs, ['scatter', 'hist', 'kde']):
    kws = dict()
    if kind == 'scatter':
        kws['c'] = cmap(0.9)
    else:
        kws['cmap'] = cmap
    psv.discrete.plot2d(
        X[:, axis], ax=ax, kind=kind,
        rms_ellipse=True, 
        rms_ellipse_kws=dict(color='red'),
        **kws
    )
    ax.format(title=kind)

In [ ]:
fig, ax = pplt.subplots()
psv.discrete.plot2d(X, ax=ax, kind='scatter', c='grey')

fractions = np.linspace(0.10, 1.0, 3)
cmap = pplt.Colormap('blues', left=0.2, reverse=True)
for fraction in fractions:
    level = ps.discrete.enclosing_ellipsoid(X[:, axis], fraction=fraction)
    psv.discrete.plot_rms_ellipse(
        X, ax=ax, level=level, 
        color=cmap(fraction), 
        lw=1.5,
        label=f'{(100.0 * fraction):.0f}%',
    )
ax.legend(loc='t', ncols=3, framealpha=0)
plt.show()

Radial density.

In [ ]:
rmin = 2.0
rmax = 3.0
_X = ps.discrete.slice_sphere(X, axis=(0, 1), rmin=rmin, rmax=rmax)

fig, ax = pplt.subplots()
psv.discrete.plot2d(X, ax=ax, kind='scatter', c='grey')
psv.discrete.plot2d(_X, ax=ax, kind='scatter', c='red')
for r in [rmin, rmax]:
    psv.circle(r, ax=ax)

In [ ]:
rmin = 1.0
rmax = 1.5
_X = ps.discrete.slice_ellipsoid(X, axis=(0, 1), rmin=rmin, rmax=rmax)

fig, ax = pplt.subplots()
psv.discrete.plot2d(X, ax=ax, kind='scatter', c='grey')
psv.discrete.plot2d(_X, ax=ax, kind='scatter', c='red')
for r in [rmin, rmax]:
    psv.discrete.plot_rms_ellipse(X, level=r, ax=ax)